Workbook

First step is to load in the data into separate pandas dataframes

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


total_gdp = pd.read_csv("total_gdp_us_inflation_adjusted.csv")
total_gdp = total_gdp.set_index('country')

gdp_percap = pd.read_csv("gdppercapita_us_inflation_adjusted.csv")
gdp_percap = gdp_percap.set_index('country')
gini = pd.read_csv("gini.csv")
gini = gini.set_index('country')

Convert values in each df from string to float, parsing special characters as needed. E.g., '400B' is 400.0e9.

In [3]:
total_gdp_copy = total_gdp.copy()
gdp_percap_copy = gdp_percap.copy()
    

for country in total_gdp_copy.index:
    for year in total_gdp_copy:
        val = total_gdp_copy.loc[country][year]
        if isinstance(val, float):
            continue    
        val = val.strip().upper()
        if val.endswith('K'):
            total_gdp_copy.loc[country,year] = float(val[:-1]) * 1e3
        elif val.endswith('M'):
            total_gdp_copy.loc[country,year] = float(val[:-1]) * 1e6
        elif val.endswith('B'):
            total_gdp_copy.loc[country,year] = float(val[:-1]) * 1e9
        elif val.endswith('TR'):
            total_gdp_copy.loc[country,year] = float(val[:-2]) * 1e12
        else:
            total_gdp_copy.loc[country,year] = float(val[:-1])

for country in gdp_percap_copy.index:
    for year in gdp_percap_copy:
        val = gdp_percap_copy.loc[country][year]
        if isinstance(val, float):
            continue    
        val = val.strip().upper()
        if val.endswith('K'):
            gdp_percap_copy.loc[country][year] = float(val[:-1]) * 1e3
        elif val.endswith('M'):
            gdp_percap_copy.loc[country][year] = float(val[:-1]) * 1e6
        elif val.endswith('B'):
            gdp_percap_copy.loc[country][year] = float(val[:-1]) * 1e9
        elif val.endswith('TR'):
            gdp_percap_copy.loc[country][year] = float(val[:-2]) * 1e12
        else:
            gdp_percap_copy.loc[country][year] = float(val[:-1])
del country
del year
del val



C:\Users\ctcas\AppData\Local\Temp\ipykernel_5352\744080966.py:29: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  gdp_percap_copy.loc[country][year] = float(val[:-1]) * 1e3
C:\Users\ctcas\AppData\Local\Temp\ipykernel_5352\744080966.py:37: Futu

Raw data loaded in as pandas dataframes. Now need to merge across a new axis called 'Metric'. Outer index is 'Metric', inner index is 'Country'. Use inner join to only include years common to all dataframes.

In [4]:
mask_total_gdp     = total_gdp_copy.index.isin(gdp_percap_copy.index) & total_gdp_copy.index.isin(gini.index)
mask_gdp_percap    = gdp_percap_copy.index.isin(total_gdp_copy.index) & gdp_percap_copy.index.isin(gini.index)
mask_gini          = gini.index.isin(total_gdp_copy.index) & gini.index.isin(gdp_percap_copy.index)

total_gdp_filtered     = total_gdp_copy[mask_total_gdp]
gdp_percap_filtered    = gdp_percap_copy[mask_gdp_percap]
gini_filtered          = gini[mask_gini]

In [5]:
combined_data = pd.concat([total_gdp_filtered, gdp_percap_filtered, gini_filtered], axis=0, keys=['Total GDP', 'GDP Per Capita', 'Gini Coeff'], join='inner')
combined_data.index.names = ['Metric', 'Country']
# new_gini = combined_data.loc['Gini Coeff']
# Canada = combined_data.xs('Canada', level='Country')
# Canada_GDP_2023 = combined_data.xs('Total GDP', level='Metric').loc['Canada', '2023']
# combined_data.shape
# totalGDP_Gini_diff = total_gdp_copy.index.difference(gini.index)
# Gini_totalGDP_diff = gini.index.difference(total_gdp.index)
# GDPpercap_Gini_diff = gdp_percap_copy.index.difference(gini.index)

# all_rejected_countries = list(totalGDP_Gini_diff) + list(Gini_totalGDP_diff)
# print(all_rejected_countries)
del mask_total_gdp
del mask_gdp_percap
del mask_gini

del total_gdp_filtered
del gdp_percap_filtered
del gini_filtered       


In [6]:
# Canada = combined_data.xs('Canada', level='Country')
# Canada_GDP_2023 = combined_data.xs('Total GDP', level='Metric').loc['Canada', '2023']

Now, remove rows (countries) that do not meet the threshold for sufficient valid data. For now, lets say we need a minimum of half of the years having a value.

In [7]:
combined_data_copy = combined_data.copy()
combined_data_copy.sort_index





<bound method DataFrame.sort_index of                          1960  1961  1962  1963  1964  1965  1966  1967  1968  \
Metric     Country                                                              
Total GDP  Afghanistan    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
           Angola         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
           Albania        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
           Andorra        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
           UAE            NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...                       ...   ...   ...   ...   ...   ...   ...   ...   ...   
Gini Coeff Samoa         40.7  40.7  40.7  40.7  40.7  40.7  40.7  40.7  40.7   
           Yemen         37.3  37.3  37.3  37.3  37.3  37.3  37.3  37.3  37.3   
           South Africa  66.0  66.9  67.5  67.9  68.0  68.1  68.2  68.4  68.5   
           Zambia        64.3  65.0  65.5  65.9  66.1  66.3  66.3  66.4

In [8]:
years_list = combined_data_copy.columns.tolist()
valid_country_list = combined_data.index.get_level_values('Country').unique().tolist()

In [9]:
# Canada = combined_data.xs('Canada', level='Country').drop()
# canada_gdp = pd.isna(Canada.loc['Total GDP'].values).sum()
# UAE = combined_data.xs('UAE', level='Country')
# UAE_gdp = pd.isna(UAE.loc['Total GDP'].values).sum()

for country in valid_country_list:
    country_slice = combined_data.xs(country, level='Country')
    country_gdp_nan_sum = pd.isna(country_slice.loc['Total GDP'].values).sum()
    country_gdp_percap_nan_sum = pd.isna(country_slice.loc['GDP Per Capita'].values).sum()
    if (country_gdp_nan_sum > len(years_list)/2) | (country_gdp_percap_nan_sum > len(years_list)/2):
        combined_data_copy = combined_data_copy.drop(country, level='Country')
        print("Dropping", country, " - ", country_gdp_nan_sum, "missing values in Total GDP.")
        print(country_gdp_percap_nan_sum, "missing values in GDP Per Capita.")

Dropping Afghanistan  -  40 missing values in Total GDP.
40 missing values in GDP Per Capita.
Dropping Djibouti  -  53 missing values in Total GDP.
53 missing values in GDP Per Capita.
Dropping Eritrea  -  44 missing values in Total GDP.
44 missing values in GDP Per Capita.
Dropping Liechtenstein  -  63 missing values in Total GDP.
63 missing values in GDP Per Capita.
Dropping Montenegro  -  37 missing values in Total GDP.
37 missing values in GDP Per Capita.
Dropping Palestine  -  34 missing values in Total GDP.
34 missing values in GDP Per Capita.
Dropping San Marino  -  38 missing values in Total GDP.
38 missing values in GDP Per Capita.
Dropping Serbia  -  35 missing values in Total GDP.
35 missing values in GDP Per Capita.
Dropping South Sudan  -  56 missing values in Total GDP.
56 missing values in GDP Per Capita.
Dropping Yemen  -  35 missing values in Total GDP.
30 missing values in GDP Per Capita.


In [10]:
valid_country_list = combined_data_copy.index.get_level_values('Country').unique().tolist()
combined_data_copy.index.names[0] == 'Metric'

True

Tasks going forward:
- User Entry (COMPLETED)
    - Prompt user to enter some number of countries, and a start year and end year.
    - Returns a tuple of ([list of Strings as country names], [list of start year, end year]).

- Dataframe cleaning: (COMPLETED)
    Remove countries that have less that length/2 valid numbers.

- Plotting:
    - Plot for each metric containing:
        - all contries and global average for time slice. 

- Presentation:
    - Summary of problem statement
    - Summary of data processing steps taken
    - Figures
    - Formatting into .pptx


In [11]:
# calculate gini dollars ( defined as GDP Per Capita * (1 - gini) )
#combined_data_copy
new_df = combined_data_copy.loc['GDP Per Capita'] * (1 - combined_data_copy.loc['Gini Coeff']/100)
new_df.round()
new_df.index = pd.MultiIndex.from_product([['Gini Dollars'], new_df.index], names=['Metric', 'Country'])

print(combined_data_copy.shape)
print(new_df.shape)
new_combined_data_copy = pd.concat([combined_data_copy, new_df])

(549, 64)
(183, 64)


In [ ]:
economic_weightclass = pd.cut(combined_data_copy.loc['Total GDP',:]['2023'], bins=[0, 1e11, 1e12, 1e13, 1e14],labels=['Lightweight','Middleweight', 'Heavyweight', 'Superpower'])
income_class = pd.cut(combined_data_copy.loc['GDP Per Capita',:]['2023'], bins=[0, 1135, 4465, 13845, 40000, 300000],labels=['Low Income','Lower Middle Income', 'Upper Middle Income', 'High Income', 'Ultra High Income'])
equality_class = pd.cut(combined_data_copy.loc['Gini Coeff',:]['2023'], bins=[0, 30, 35, 40, 45, 50, 100],labels=['Very Low Inequality','Low Inequality', 'Low-Moderate Inequality', 'Moderate Inequality', 'Moderate-High Inequality', 'High Inequality'])
livability_class = pd.cut(new_combined_data_copy.loc['Gini Dollars',:]['2023'], bins=[0, 1135, 4465, 13845, 30000, 300000],labels=['Low Livability','Low-Moderate Livability', 'Moderate Livability', 'Moderate-High Livability', 'High Livability'])

#classification = pd.concat([economic_weightclass, income_class, equality_class, livability_class])
classification = pd.concat([economic_weightclass,economic_weightclass,economic_weightclass,economic_weightclass]) #ADDED BY CONNER


In [ ]:
metrics = new_combined_data_copy.index.get_level_values('Metric')
countries = new_combined_data_copy.index.get_level_values('Country')

# Use the classification series you created
new_index = pd.MultiIndex.from_arrays([
    metrics,
    classification.values,  # classification per (Metric, Country)
    countries
], names=['Metric', 'Classification', 'Country'])

# Assign to the DataFrame
new_combined_data_copy.index = new_index

ndf = new_combined_data_copy

ndf.pivot_table('2023',index='Metric',columns='Classification') #PIVOT TABLE BASED ON ECONOMIC WEIGHTCLASS


C:\Users\ctcas\AppData\Local\Temp\ipykernel_5352\783039096.py:17: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  ndf.pivot_table('2023',index='Metric',columns='Classification')


Classification,Lightweight,Middleweight,Heavyweight,Superpower
Metric,,,,
GDP Per Capita,8015.470085,24412.911111,27164.6,39050.0
Gini Coeff,41.174359,37.295556,40.066667,42.5
Gini Dollars,5130.431684,16029.0852,17158.870267,21540.85
Total GDP,26467930769.23077,362488888888.888916,2215333333333.333496,19650000000000.0
